In [ ]:
#importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
import glob
import pyarrow.parquet as pq
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import math
import re
import string
import pyarrow as pa
import os
import math
import seaborn as sb
from google.colab import drive
drive.mount('/content/gdrive')
#!unzip /content/gdrive/MyDrive/data.zip -d /content/gdrive/MyDrive/Etsy_Data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#setting path and loading the csv file
PATH = f"/content/gdrive/MyDrive/Etsy_Data/data"
p_f = f'{PATH}/parquet/train'
test = pd.read_parquet(p_f,engine='pyarrow')

In [ ]:
test.columns

Index(['product_id', 'title', 'description', 'tags', 'type', 'room',
       'craft_type', 'recipient', 'material', 'occasion', 'holiday',
       'art_subject', 'style', 'shape', 'pattern', 'bottom_category_id',
       'bottom_category_text', 'top_category_id', 'top_category_text',
       'color_id', 'color_text'],
      dtype='object')

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245485 entries, 0 to 245484
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   product_id            245485 non-null  int64 
 1   title                 244545 non-null  object
 2   description           244545 non-null  object
 3   tags                  210575 non-null  object
 4   type                  244211 non-null  object
 5   room                  8727 non-null    object
 6   craft_type            32520 non-null   object
 7   recipient             13753 non-null   object
 8   material              20876 non-null   object
 9   occasion              53229 non-null   object
 10  holiday               41019 non-null   object
 11  art_subject           2773 non-null    object
 12  style                 17032 non-null   object
 13  shape                 2358 non-null    object
 14  pattern               10678 non-null   object
 15  bottom_category_i

## Dropping unnecessary columns
Removed the columns such as tags, type, room etc since there were more than 50% missing values which were not imputable and columns were of no use.
Also removed the type column since it is not giving to much information for the analysis.

In [ ]:
#Dropping the columns which are not required
test = test.drop(['type', 'room',
       'craft_type', 'recipient', 'material', 'occasion', 'holiday',
       'art_subject', 'style', 'shape', 'pattern'],axis = 1)

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245485 entries, 0 to 245484
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   product_id            245485 non-null  int64 
 1   title                 244545 non-null  object
 2   description           244545 non-null  object
 3   tags                  210575 non-null  object
 4   bottom_category_id    245485 non-null  int64 
 5   bottom_category_text  245485 non-null  object
 6   top_category_id       245485 non-null  int64 
 7   top_category_text     245485 non-null  object
 8   color_id              245485 non-null  int64 
 9   color_text            245485 non-null  object
dtypes: int64(4), object(6)
memory usage: 18.7+ MB


### Basic Cleaning of Text Data


In [ ]:
test['title'] = test['title'].fillna("")
test['title'] = test['title'].apply(lambda x: x.lower() if pd.notna(x) else x)
test['title'] = test['title'].str.replace(r'\\n','')
test['title'] = test['title'].str.replace('[^A-Za-z\s]','')
test['title'] = test['title'].str.replace('\d+','')
test['title'] = test['title'].str.replace(r'http\S+|www.\S+', '')

test['description'] = test['description'].fillna("")
test['description'] = test['description'].apply(lambda x: x.lower() if pd.notna(x) else x)
test['description'] = test['description'].str.replace(r'\\n','')
test['description'] = test['description'].str.replace('[^A-Za-z\s]','')
test['description'] = test['description'].str.replace('\d+','')
test['description'] = test['description'].str.replace(r'http\S+|www.\S+', '')

test['tags'] = test['tags'].fillna("")
test['tags'] = test['tags'].apply(lambda x: x.lower() if pd.notna(x) else x)
test['tags'] = test['tags'].str.replace(r'\\n','')
test['tags'] = test['tags'].str.replace('[^A-Za-z\s]','')
test['tags'] = test['tags'].str.replace('\d+','')
test['tags'] = test['tags'].str.replace(r'http\S+|www.\S+', '')

<ipython-input-17-e350f8db3654>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test['title'] = test['title'].str.replace(r'\\n','')
<ipython-input-17-e350f8db3654>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  test['title'] = test['title'].str.replace('[^A-Za-z\s]','')
<ipython-input-17-e350f8db3654>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  test['title'] = test['title'].str.replace('\d+','')
<ipython-input-17-e350f8db3654>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  test['title'] = test['title'].str.replace(r'http\S+|www.\S+', '')
<ipython-input-17-e350f8db3654>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  test['description'] = test['description'].str.replace(r'\\n','')
<ipython-input-17-e350f8db3654>:11: FutureWarnin

### Remove Stop Words from the data

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
def remove_stop_words(text):
    if pd.isna(text):
        return ''
    words = text.split()
    filtered_words = [word for word in words if len(word) > 2 and word not in stop_words]
    return ' '.join(filtered_words)

test['title'] = test['title'].apply(remove_stop_words)
test['description'] = test['description'].apply(remove_stop_words)
test['tags'] = test['tags'].apply(remove_stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Lemmatise the Text

In [ ]:
lemmatizer = WordNetLemmatizer()

def wordnet_pos(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('R'):
            return wordnet.ADV
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        else:
            return wordnet.NOUN

def lemmatization_of_text(text):
    words = word_tokenize(text)
    pos_tags = nltk.pos_tag(words)
    lemmatized_words = [lemmatizer.lemmatize(word, wordnet_pos(tag)) for word, tag in pos_tags]
    lemmatized_text = ' '.join(lemmatized_words)
    return lemmatized_text

In [ ]:
test['description']=test['description'].apply(lemmatization_of_text)
test['title']=test['title'].apply(lemmatization_of_text)
test['tags']=test['tags'].apply(lemmatization_of_text)

In [ ]:
test['combine'] = test['title'] + ' ' + test['description'] + ' ' + test['tags']

In [ ]:
test

,product_id,title,description,tags,bottom_category_id,bottom_category_text,top_category_id,top_category_text,color_id,color_text,combine
0,543497833,fullface custom motorcycle helmet motorcycle s...,helmetartthai thailand since name phayong khon...,predator helmetmotorcycle helmethelmethandmade...,2804,accessories.hats_and_caps.helmets.sports_helme...,0,accessories,12,purple,fullface custom motorcycle helmet motorcycle s...
1,718680498,fullface predator motorcycle helmet custom pre...,helmetartthai thailand since name phayong khon...,predator helmetcustom helmethandmade helmepain...,2804,accessories.hats_and_caps.helmets.sports_helme...,0,accessories,1,black,fullface predator motorcycle helmet custom pre...
2,718823736,fullface custom motorcycle helmet motorcycle s...,helmetartthai thailand since name phayong khon...,predator helmetcustom helmethandmade helmetair...,2804,accessories.hats_and_caps.helmets.sports_helme...,0,accessories,2,blue,fullface custom motorcycle helmet motorcycle s...
3,608840803,fullface custom motorcycle helmet motorcycle s...,helmetartthai thailand since name phayong khon...,predator helmetcustom helmethandmade helmethel...,2804,accessories.hats_and_caps.helmets.sports_helme...,0,accessories,4,brown,fullface custom motorcycle helmet motorcycle s...
4,804070543,fullface predator motorcycle helmet custom pre...,helmetartthai thailand since name phayong khon...,custom helmethandmade helmetpredator helmethel...,2804,accessories.hats_and_caps.helmets.sports_helme...,0,accessories,1,black,fullface predator motorcycle helmet custom pre...
...,...,...,...,...,...,...,...,...,...,...,...
245480,1330991677,vintage apri night black alumesh evening bag c...,charm vintage apri night black alumesh evening...,black mesh beadingapri nightsevening bagcrossb...,157,bags_and_purses.handbags.clutches_and_evening_...,2,bags_and_purses,1,black,vintage apri night black alumesh evening bag c...
245481,927998323,gold eloquent envelope clutch bag,look classy posh bag hit hot club gold envelop...,clutchgoldsoftenvelopedisco,157,bags_and_purses.handbags.clutches_and_evening_...,2,bags_and_purses,7,gold,gold eloquent envelope clutch bag look classy ...
245482,1393052238,personalize pouch toiletry bag makeup bag,personalize item return exchange accordance ge...,,157,bags_and_purses.handbags.clutches_and_evening_...,2,bags_and_purses,17,white,personalize pouch toiletry bag makeup bag pers...
245483,775199837,floral handbag canvas bag bridesmaid bag guest...,handbag flower jute ideal day partiesdetails c...,flower handbagcanvas bagbridesmaid bagguest ba...,157,bags_and_purses.handbags.clutches_and_evening_...,2,bags_and_purses,0,beige,floral handbag canvas bag bridesmaid bag guest...


Converted the dataframe into csv file and stored it on Drive for easy usage

In [ ]:
test.to_csv(r'/content/gdrive/MyDrive/Etsy_Data/Clean_text_data.csv', index=False)

In [ ]:
test = pd.read_csv('/content/gdrive/MyDrive/Etsy_Data/Clean_text_data.csv')

In [ ]:
test['bottom_category_id']

array([ 2804,  1406,  1515, ..., 11222, 12140,   157])

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245485 entries, 0 to 245484
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   product_id            245485 non-null  int64 
 1   title                 244493 non-null  object
 2   description           244496 non-null  object
 3   tags                  210536 non-null  object
 4   bottom_category_id    245485 non-null  int64 
 5   bottom_category_text  245485 non-null  object
 6   top_category_id       245485 non-null  int64 
 7   top_category_text     245485 non-null  object
 8   color_id              245485 non-null  int64 
 9   color_text            245485 non-null  object
 10  combine               245485 non-null  object
dtypes: int64(4), object(7)
memory usage: 20.6+ MB


## Models for top category prediction

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score

batch_size = 20000
num_batches = len(test) // batch_size + (len(test) % batch_size > 0)
model_top = LogisticRegression(max_iter=1000)

vectorizer = CountVectorizer(max_features=10000)
y_true_train = []
y_pred_train = []
y_true_val = []
y_pred_val = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(test))
    X_text = test['combine'][start_idx:end_idx]
    y = test['top_category_id'][start_idx:end_idx]

    X = vectorizer.fit_transform(X_text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model_top.fit(X_train,y_train)

    y_pred_train_batch = model_top.predict(X_train)
    y_true_train.extend(y_train)
    y_pred_train.extend(y_pred_train_batch)

    y_pred_val_batch = model_top.predict(X_test)
    y_true_val.extend(y_test)
    y_pred_val.extend(y_pred_val_batch)

    train_recall = recall_score(y_true_train, y_pred_train, average='weighted')
    train_f1_score = f1_score(y_true_train, y_pred_train, average='weighted')
    train_accuracy = accuracy_score(y_true_train, y_pred_train)

    val_recall = recall_score(y_true_val, y_pred_val, average='weighted')
    val_f1_score = f1_score(y_true_val, y_pred_val, average='weighted')
    val_accuracy = accuracy_score(y_true_val, y_pred_val)

print("Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(train_recall, train_f1_score, train_accuracy))
print("Val Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(val_recall, val_f1_score, val_accuracy))

Top Category - Recall: 0.9966, F1 Score: 0.9966, Accuracy: 0.9966
Val Top Category - Recall: 0.8948, F1 Score: 0.8945, Accuracy: 0.8948


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score

batch_size = 20000
num_batches = len(test) // batch_size + (len(test) % batch_size > 0)
model = MultinomialNB()

vectorizer = CountVectorizer(max_features=10000)
y_true_train = []
y_pred_train = []
y_true_val = []
y_pred_val = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(test))
    X_text = test['combine'][start_idx:end_idx]
    y = test['top_category_id'][start_idx:end_idx]

    X = vectorizer.fit_transform(X_text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train,y_train)

    y_pred_train_batch = model.predict(X_train)
    y_true_train.extend(y_train)
    y_pred_train.extend(y_pred_train_batch)

    y_pred_val_batch = model.predict(X_test)
    y_true_val.extend(y_test)
    y_pred_val.extend(y_pred_val_batch)

    train_recall = recall_score(y_true_train, y_pred_train, average='weighted')
    train_f1_score = f1_score(y_true_train, y_pred_train, average='weighted')
    train_accuracy = accuracy_score(y_true_train, y_pred_train)


    val_recall = recall_score(y_true_val, y_pred_val, average='weighted')
    val_f1_score = f1_score(y_true_val, y_pred_val, average='weighted')
    val_accuracy = accuracy_score(y_true_val, y_pred_val)

print("Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(train_recall, train_f1_score, train_accuracy))
print("Val Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(val_recall, val_f1_score, val_accuracy))



Top Category - Recall: 0.9031, F1 Score: 0.9031, Accuracy: 0.9031
Val Top Category - Recall: 0.8622, F1 Score: 0.8622, Accuracy: 0.8622


In [ ]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score

batch_size = 20000
num_batches = len(test) // batch_size + (len(test) % batch_size > 0)
model = SVC()

vectorizer = CountVectorizer()
y_true_train = []
y_pred_train = []
y_true_val = []
y_pred_val = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(test))
    X_text = test['combine'][start_idx:end_idx]
    y = test['top_category_id'][start_idx:end_idx]

    X = vectorizer.fit_transform(X_text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train,y_train)

    y_pred_train_batch = model.predict(X_train)
    y_true_train.extend(y_train)
    y_pred_train.extend(y_pred_train_batch)

    y_pred_val_batch = model.predict(X_test)
    y_true_val.extend(y_test)
    y_pred_val.extend(y_pred_val_batch)

    train_recall = recall_score(y_true_train, y_pred_train, average='weighted')
    train_f1_score = f1_score(y_true_train, y_pred_train, average='weighted')
    train_accuracy = accuracy_score(y_true_train, y_pred_train)

    val_recall = recall_score(y_true_val, y_pred_val, average='weighted')
    val_f1_score = f1_score(y_true_val, y_pred_val, average='weighted')
    val_accuracy = accuracy_score(y_true_val, y_pred_val)

print("Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(train_recall, train_f1_score, train_accuracy))
print("Val Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(val_recall, val_f1_score, val_accuracy))

Top Category - Recall: 0.9310, F1 Score: 0.9309, Accuracy: 0.9310
Val Top Category - Recall: 0.8394, F1 Score: 0.8386, Accuracy: 0.8394


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score

batch_size = 20000
num_batches = len(test) // batch_size + (len(test) % batch_size > 0)
model = DecisionTreeClassifier(random_state=0)

vectorizer = CountVectorizer()
y_true_train = []
y_pred_train = []
y_true_val = []
y_pred_val = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(test))
    X_text = test['combine'][start_idx:end_idx]
    y = test['top_category_id'][start_idx:end_idx]

    X = vectorizer.fit_transform(X_text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train,y_train)

    y_pred_train_batch = model.predict(X_train)
    y_true_train.extend(y_train)
    y_pred_train.extend(y_pred_train_batch)

    y_pred_val_batch = model.predict(X_test)
    y_true_val.extend(y_test)
    y_pred_val.extend(y_pred_val_batch)

    train_recall = recall_score(y_true_train, y_pred_train, average='weighted')
    train_f1_score = f1_score(y_true_train, y_pred_train, average='weighted')
    train_accuracy = accuracy_score(y_true_train, y_pred_train)

    val_recall = recall_score(y_true_val, y_pred_val, average='weighted')
    val_f1_score = f1_score(y_true_val, y_pred_val, average='weighted')
    val_accuracy = accuracy_score(y_true_val, y_pred_val)

print("Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(train_recall, train_f1_score, train_accuracy))
print("Val Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(val_recall, val_f1_score, val_accuracy))

Top Category - Recall: 0.9972, F1 Score: 0.9972, Accuracy: 0.9972
Val Top Category - Recall: 0.7425, F1 Score: 0.7414, Accuracy: 0.7425


## Models for Bottom Category Prediction

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score

batch_size = 15000
num_batches = len(test) // batch_size + (len(test) % batch_size > 0)
model_bottom = LogisticRegression(penalty='l1',solver='saga',max_iter=1000)

vectorizer = CountVectorizer(max_features=10000)
y_true_train = []
y_pred_train = []
y_true_val = []
y_pred_val = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(test))
    X_text = test['combine'][start_idx:end_idx]
    y = test['bottom_category_id'][start_idx:end_idx]

    X = vectorizer.fit_transform(X_text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model_bottom.fit(X_train,y_train)

    y_pred_train_batch = model_bottom.predict(X_train)
    y_true_train.extend(y_train)
    y_pred_train.extend(y_pred_train_batch)

    y_pred_val_batch = model_bottom.predict(X_test)
    y_true_val.extend(y_test)
    y_pred_val.extend(y_pred_val_batch)

    train_recall = recall_score(y_true_train, y_pred_train, average='weighted')
    train_f1_score = f1_score(y_true_train, y_pred_train, average='weighted')
    train_accuracy = accuracy_score(y_true_train, y_pred_train)

    val_recall = recall_score(y_true_val, y_pred_val, average='weighted')
    val_f1_score = f1_score(y_true_val, y_pred_val, average='weighted')
    val_accuracy = accuracy_score(y_true_val, y_pred_val)

print("Bottom Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(train_recall, train_f1_score, train_accuracy))
print("Val Bottom Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(val_recall, val_f1_score, val_accuracy))

KeyboardInterrupt: ignored

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score

batch_size = 20000
num_batches = len(test) // batch_size + (len(test) % batch_size > 0)
model = MultinomialNB()

vectorizer = CountVectorizer()
y_true_train = []
y_pred_train = []
y_true_val = []
y_pred_val = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(test))
    X_text = test['combine'][start_idx:end_idx]
    y = test['bottom_category_id'][start_idx:end_idx]

    X = vectorizer.fit_transform(X_text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train,y_train)

    y_pred_train_batch = model.predict(X_train)
    y_true_train.extend(y_train)
    y_pred_train.extend(y_pred_train_batch)

    y_pred_val_batch = model.predict(X_test)
    y_true_val.extend(y_test)
    y_pred_val.extend(y_pred_val_batch)

    train_recall = recall_score(y_true_train, y_pred_train, average='weighted')
    train_f1_score = f1_score(y_true_train, y_pred_train, average='weighted')
    train_accuracy = accuracy_score(y_true_train, y_pred_train)


    val_recall = recall_score(y_true_val, y_pred_val, average='weighted')
    val_f1_score = f1_score(y_true_val, y_pred_val, average='weighted')
    val_accuracy = accuracy_score(y_true_val, y_pred_val)

print("Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(train_recall, train_f1_score, train_accuracy))
print("Val Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(val_recall, val_f1_score, val_accuracy))

Top Category - Recall: 0.9408, F1 Score: 0.9429, Accuracy: 0.9408
Val Top Category - Recall: 0.7379, F1 Score: 0.7456, Accuracy: 0.7379


In [ ]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score

batch_size = 20000
num_batches = len(test) // batch_size + (len(test) % batch_size > 0)
model = SVC()

vectorizer = CountVectorizer()
y_true_train = []
y_pred_train = []
y_true_val = []
y_pred_val = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(test))
    X_text = test['combine'][start_idx:end_idx]
    y = test['bottom_category_id'][start_idx:end_idx]

    X = vectorizer.fit_transform(X_text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train,y_train)

    y_pred_train_batch = model.predict(X_train)
    y_true_train.extend(y_train)
    y_pred_train.extend(y_pred_train_batch)

    y_pred_val_batch = model.predict(X_test)
    y_true_val.extend(y_test)
    y_pred_val.extend(y_pred_val_batch)

    train_recall = recall_score(y_true_train, y_pred_train, average='weighted')
    train_f1_score = f1_score(y_true_train, y_pred_train, average='weighted')
    train_accuracy = accuracy_score(y_true_train, y_pred_train)

    val_recall = recall_score(y_true_val, y_pred_val, average='weighted')
    val_f1_score = f1_score(y_true_val, y_pred_val, average='weighted')
    val_accuracy = accuracy_score(y_true_val, y_pred_val)

print("Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(train_recall, train_f1_score, train_accuracy))
print("Val Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(val_recall, val_f1_score, val_accuracy))

Top Category - Recall: 0.8970, F1 Score: 0.9193, Accuracy: 0.8970
Val Top Category - Recall: 0.7095, F1 Score: 0.7519, Accuracy: 0.7095


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score

batch_size = 20000
num_batches = len(test) // batch_size + (len(test) % batch_size > 0)
model = DecisionTreeClassifier(random_state=0)

vectorizer = CountVectorizer()
y_true_train = []
y_pred_train = []
y_true_val = []
y_pred_val = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(test))
    X_text = test['combine'][start_idx:end_idx]
    y = test['bottom_category_id'][start_idx:end_idx]

    X = vectorizer.fit_transform(X_text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train,y_train)

    y_pred_train_batch = model.predict(X_train)
    y_true_train.extend(y_train)
    y_pred_train.extend(y_pred_train_batch)

    y_pred_val_batch = model.predict(X_test)
    y_true_val.extend(y_test)
    y_pred_val.extend(y_pred_val_batch)

    train_recall = recall_score(y_true_train, y_pred_train, average='weighted')
    train_f1_score = f1_score(y_true_train, y_pred_train, average='weighted')
    train_accuracy = accuracy_score(y_true_train, y_pred_train)

    val_recall = recall_score(y_true_val, y_pred_val, average='weighted')
    val_f1_score = f1_score(y_true_val, y_pred_val, average='weighted')
    val_accuracy = accuracy_score(y_true_val, y_pred_val)

print("Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(train_recall, train_f1_score, train_accuracy))
print("Val Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(val_recall, val_f1_score, val_accuracy))

Top Category - Recall: 0.9968, F1 Score: 0.9972, Accuracy: 0.9968
Val Top Category - Recall: 0.6869, F1 Score: 0.6873, Accuracy: 0.6869


## Model For Color Id

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score

batch_size = 20000
num_batches = len(test) // batch_size + (len(test) % batch_size > 0)
model_color = LogisticRegression(max_iter=1000)

vectorizer = CountVectorizer(max_features=10000)
y_true_train = []
y_pred_train = []
y_true_val = []
y_pred_val = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(test))
    X_text = test['combine'][start_idx:end_idx]
    y = test['color_id'][start_idx:end_idx]

    X = vectorizer.fit_transform(X_text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model_color.fit(X_train,y_train)

    y_pred_train_batch = model_color.predict(X_train)
    y_true_train.extend(y_train)
    y_pred_train.extend(y_pred_train_batch)

    y_pred_val_batch = model_color.predict(X_test)
    y_true_val.extend(y_test)
    y_pred_val.extend(y_pred_val_batch)

    train_recall = recall_score(y_true_train, y_pred_train, average='weighted')
    train_f1_score = f1_score(y_true_train, y_pred_train, average='weighted')
    train_accuracy = accuracy_score(y_true_train, y_pred_train)

    val_recall = recall_score(y_true_val, y_pred_val, average='weighted')
    val_f1_score = f1_score(y_true_val, y_pred_val, average='weighted')
    val_accuracy = accuracy_score(y_true_val, y_pred_val)

print("Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(train_recall, train_f1_score, train_accuracy))
print("Val Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(val_recall, val_f1_score, val_accuracy))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score

batch_size = 20000
num_batches = len(test) // batch_size + (len(test) % batch_size > 0)
model_color = MultinomialNB()

vectorizer = CountVectorizer()
y_true_train = []
y_pred_train = []
y_true_val = []
y_pred_val = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(test))
    X_text = test['combine'][start_idx:end_idx]
    y = test['color_id'][start_idx:end_idx]

    X = vectorizer.fit_transform(X_text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model_color.fit(X_train,y_train)

    y_pred_train_batch = model_color.predict(X_train)
    y_true_train.extend(y_train)
    y_pred_train.extend(y_pred_train_batch)

    y_pred_val_batch = model_color.predict(X_test)
    y_true_val.extend(y_test)
    y_pred_val.extend(y_pred_val_batch)

    train_recall = recall_score(y_true_train, y_pred_train, average='weighted')
    train_f1_score = f1_score(y_true_train, y_pred_train, average='weighted')
    train_accuracy = accuracy_score(y_true_train, y_pred_train)


    val_recall = recall_score(y_true_val, y_pred_val, average='weighted')
    val_f1_score = f1_score(y_true_val, y_pred_val, average='weighted')
    val_accuracy = accuracy_score(y_true_val, y_pred_val)

print("Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(train_recall, train_f1_score, train_accuracy))
print("Val Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(val_recall, val_f1_score, val_accuracy))

Top Category - Recall: 0.5858, F1 Score: 0.5457, Accuracy: 0.5858
Val Top Category - Recall: 0.3426, F1 Score: 0.2976, Accuracy: 0.3426


In [ ]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score

batch_size = 20000
num_batches = len(test) // batch_size + (len(test) % batch_size > 0)
model = SVC()

vectorizer = CountVectorizer()
y_true_train = []
y_pred_train = []
y_true_val = []
y_pred_val = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(test))
    X_text = test['combine'][start_idx:end_idx]
    y = test['color_id'][start_idx:end_idx]

    X = vectorizer.fit_transform(X_text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train,y_train)

    y_pred_train_batch = model.predict(X_train)
    y_true_train.extend(y_train)
    y_pred_train.extend(y_pred_train_batch)

    y_pred_val_batch = model.predict(X_test)
    y_true_val.extend(y_test)
    y_pred_val.extend(y_pred_val_batch)

    train_recall = recall_score(y_true_train, y_pred_train, average='weighted')
    train_f1_score = f1_score(y_true_train, y_pred_train, average='weighted')
    train_accuracy = accuracy_score(y_true_train, y_pred_train)

    val_recall = recall_score(y_true_val, y_pred_val, average='weighted')
    val_f1_score = f1_score(y_true_val, y_pred_val, average='weighted')
    val_accuracy = accuracy_score(y_true_val, y_pred_val)

print("Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(train_recall, train_f1_score, train_accuracy))
print("Val Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(val_recall, val_f1_score, val_accuracy))

Top Category - Recall: 0.7035, F1 Score: 0.6988, Accuracy: 0.7035
Val Top Category - Recall: 0.4467, F1 Score: 0.4292, Accuracy: 0.4467


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score

batch_size = 20000
num_batches = len(test) // batch_size + (len(test) % batch_size > 0)
model = DecisionTreeClassifier(random_state=0)

vectorizer = CountVectorizer()
y_true_train = []
y_pred_train = []
y_true_val = []
y_pred_val = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(test))
    X_text = test['combine'][start_idx:end_idx]
    y = test['color_id'][start_idx:end_idx]

    X = vectorizer.fit_transform(X_text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train,y_train)

    y_pred_train_batch = model.predict(X_train)
    y_true_train.extend(y_train)
    y_pred_train.extend(y_pred_train_batch)

    y_pred_val_batch = model.predict(X_test)
    y_true_val.extend(y_test)
    y_pred_val.extend(y_pred_val_batch)

    train_recall = recall_score(y_true_train, y_pred_train, average='weighted')
    train_f1_score = f1_score(y_true_train, y_pred_train, average='weighted')
    train_accuracy = accuracy_score(y_true_train, y_pred_train)

    val_recall = recall_score(y_true_val, y_pred_val, average='weighted')
    val_f1_score = f1_score(y_true_val, y_pred_val, average='weighted')
    val_accuracy = accuracy_score(y_true_val, y_pred_val)

print("Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(train_recall, train_f1_score, train_accuracy))
print("Val Top Category - Recall: {:.4f}, F1 Score: {:.4f}, Accuracy: {:.4f}".format(val_recall, val_f1_score, val_accuracy))

Top Category - Recall: 0.9945, F1 Score: 0.9945, Accuracy: 0.9945
Val Top Category - Recall: 0.4303, F1 Score: 0.4280, Accuracy: 0.4303


## Testing Set

In [ ]:
#setting path and loading the csv file
PATH = f"/content/gdrive/MyDrive/Etsy_Data/data"
p_f = f'{PATH}/parquet/test'
final = pd.read_parquet(p_f,engine='pyarrow')

In [ ]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27119 entries, 0 to 27118
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   product_id   27119 non-null  int64 
 1   title        27008 non-null  object
 2   description  27008 non-null  object
 3   tags         23232 non-null  object
 4   type         26975 non-null  object
 5   room         958 non-null    object
 6   craft_type   3606 non-null   object
 7   recipient    1478 non-null   object
 8   material     2341 non-null   object
 9   occasion     5766 non-null   object
 10  holiday      4546 non-null   object
 11  art_subject  278 non-null    object
 12  style        1878 non-null   object
 13  shape        302 non-null    object
 14  pattern      1170 non-null   object
dtypes: int64(1), object(14)
memory usage: 3.1+ MB


In [ ]:
final = final.drop(['type', 'room',
       'craft_type', 'recipient', 'material', 'occasion', 'holiday',
       'art_subject', 'style', 'shape', 'pattern'],axis = 1)

In [ ]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27119 entries, 0 to 27118
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   product_id   27119 non-null  int64 
 1   title        27008 non-null  object
 2   description  27008 non-null  object
 3   tags         23232 non-null  object
dtypes: int64(1), object(3)
memory usage: 847.6+ KB


In [ ]:
final['title'] = final['title'].fillna("")
final['title'] = final['title'].apply(lambda x: x.lower() if pd.notna(x) else x)
final['title'] = final['title'].str.replace(r'\\n','')
final['title'] = final['title'].str.replace('[^A-Za-z\s]','')
final['title'] = final['title'].str.replace('\d+','')
final['title'] = final['title'].str.replace(r'http\S+|www.\S+', '')

final['description'] = final['description'].fillna("")
final['description'] = final['description'].apply(lambda x: x.lower() if pd.notna(x) else x)
final['description'] = final['description'].str.replace(r'\\n','')
final['description'] = final['description'].str.replace('[^A-Za-z\s]','')
final['description'] = final['description'].str.replace('\d+','')
final['description'] = final['description'].str.replace(r'http\S+|www.\S+', '')

final['tags'] = final['tags'].fillna("")
final['tags'] = final['tags'].apply(lambda x: x.lower() if pd.notna(x) else x)
final['tags'] = final['tags'].str.replace(r'\\n','')
final['tags'] = final['tags'].str.replace('[^A-Za-z\s]','')
final['tags'] = final['tags'].str.replace('\d+','')
final['tags'] = final['tags'].str.replace(r'http\S+|www.\S+', '')

<ipython-input-9-4dbaf3847965>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  final['title'] = final['title'].str.replace(r'\\n','')
<ipython-input-9-4dbaf3847965>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  final['title'] = final['title'].str.replace('[^A-Za-z\s]','')
<ipython-input-9-4dbaf3847965>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  final['title'] = final['title'].str.replace('\d+','')
<ipython-input-9-4dbaf3847965>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  final['title'] = final['title'].str.replace(r'http\S+|www.\S+', '')
<ipython-input-9-4dbaf3847965>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  final['description'] = final['description'].str.replace(r'\\n','')
<ipython-input-9-4dbaf3847965>:11: FutureWa

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
def remove_stop_words(text):
    if pd.isna(text):
        return ''
    words = text.split()
    filtered_words = [word for word in words if len(word) > 2 and word not in stop_words]
    return ' '.join(filtered_words)

final['title'] = final['title'].apply(remove_stop_words)
final['description'] = final['description'].apply(remove_stop_words)
final['tags'] = final['tags'].apply(remove_stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

def wordnet_pos(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('R'):
            return wordnet.ADV
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        else:
            return wordnet.NOUN

def lemmatization_of_text(text):
    words = word_tokenize(text)
    pos_tags = nltk.pos_tag(words)
    lemmatized_words = [lemmatizer.lemmatize(word, wordnet_pos(tag)) for word, tag in pos_tags]
    lemmatized_text = ' '.join(lemmatized_words)
    return lemmatized_text

In [ ]:
final['description']=final['description'].apply(lemmatization_of_text)
final['title']=final['title'].apply(lemmatization_of_text)
final['tags']=final['tags'].apply(lemmatization_of_text)

In [ ]:
final['combine'] = final['title'] + ' ' + final['description'] + ' ' + final['tags']

In [ ]:
final

,product_id,title,description,tags,combine
0,1323824090,small press reddishorange japanese maple leave...,small press reddishorange japanese maple leave...,leavesartcraftspaper craftsfallsoapbookmarksre...,small press reddishorange japanese maple leave...
1,544175230,christmas artificial leaf holiday decor glitte...,glitter fake leavesprice leaf leaf size quot q...,holiday decorglitter fake leavesfaux greeneryf...,christmas artificial leaf holiday decor glitte...
2,895779370,fresh holly leave count christmas greenery,list include fresh holly leave freshly pick pi...,lucky holly leaveschristmas greenerychristmas ...,fresh holly leave count christmas greenery lis...
3,641102090,christmas artificial greenery gold plant glitt...,glitter pine stemsprice branchthe total height...,glitter pine stemsholiday decorchristmas craft...,christmas artificial greenery gold plant glitt...
4,1340603160,red crown thorn corona thai hybrid sale cut root,free shipping,cactus live plantcrown thorn,red crown thorn corona thai hybrid sale cut ro...
...,...,...,...,...,...
27114,809100810,book thing george stimpson,vintage book thousand thing george stimpson fi...,,book thing george stimpson vintage book thousa...
27115,1358895140,vintage story sugar hawaii softcover book hawa...,story sugar hawaiipublished hawaiian sugar pla...,vintagehawaiisugarhistoryhawaiianfirst edition...,vintage story sugar hawaii softcover book hawa...
27116,1366768240,francis king man rock vintage penguin book fir...,man rock seventh novel francis king hail beryl...,penguin book penguinbook lover gift classic bo...,francis king man rock vintage penguin book fir...
27117,1168372040,child abbey regina roche three volume one illu...,charm early victorian antique edition child ab...,child abbeyregina maria rocheregina rochevicto...,child abbey regina roche three volume one illu...


In [ ]:
#final.to_csv(r'/content/gdrive/MyDrive/Etsy_Data/final_testing_clean_data.csv', index=False)

final = pd.read_csv('/content/gdrive/MyDrive/Etsy_Data/final_testing_clean_data.csv')

In [ ]:
vectorizer = CountVectorizer(max_features=10000)

batch_size = 20000
num_batches = len(final) // batch_size + (len(final) % batch_size > 0)
test_pred = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(final))
    Y = final['combine'][start_idx:end_idx]
    Y = vectorizer.fit_transform(final['combine'])
    test_pred_top = model_top.predict(Y)

In [ ]:
vectorizer = CountVectorizer(max_features=66691)

batch_size = 20000
num_batches = len(final) // batch_size + (len(final) % batch_size > 0)
test_pred = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(final))
    Y = final['combine'][start_idx:end_idx]
    Y = vectorizer.fit_transform(final['combine'])
    test_pred = model.predict(Y)

In [ ]:
vectorizer = CountVectorizer(max_features=66691)

batch_size = 20000
num_batches = len(final) // batch_size + (len(final) % batch_size > 0)
test_pred = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(final))
    Y = final['combine'][start_idx:end_idx]
    Y = vectorizer.fit_transform(final['combine'])
    test_pred = model_color.predict(Y)

In [ ]:
test_pred_top = pd.DataFrame(test_pred_top)
#test_pred = final['product_id'] + test_pred
test_pred = pd.concat([test_pred_top,final['product_id']],axis=1)

In [ ]:
test_pred.to_csv(r'/content/gdrive/MyDrive/Etsy_Data/top_category_prediction_22266982.csv',index=False)

In [ ]:
test_pred = pd.DataFrame(test_pred)
#test_pred = final['product_id'] + test_pred
test_pred = pd.concat([test_pred,final['product_id']],axis=1)

In [ ]:
test_pred.to_csv(r'/content/gdrive/MyDrive/Etsy_Data/bottom_category_prediction_22266982.csv',index=False)

In [ ]:
test_pred = pd.DataFrame(test_pred)
#test_pred = final['product_id'] + test_pred
test_pred = pd.concat([test_pred,final['product_id']],axis=1)
test_pred.to_csv(r'/content/gdrive/MyDrive/Etsy_Data/color_category_prediction_22266982.csv',index=False)